In [57]:
%load_ext autoreload
%autoreload 2
import cv2
import numpy as np
import math
import imageio
import glob2 as glob
import os
import shutil
import logging

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
pathToVideos = "/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/"

In [59]:
def getAllDirectories (pathToDirectory):
    '''
        Function that gets all the subdirectories of a folder
        
        Args:
            path: Path to root directory
            
        Returns: List of all the complete paths to files
    '''
    files = []
    files = glob.glob(pathToDirectory + '*/', recursive=True)
    return files

def getAllFiles (pathToFiles):
    '''
        Function that gets all the filenames from a directory
        
        Args:
            path: Path to directory
            
        Returns: List of all the complete paths to files
    '''
    files = []
    print(pathToFiles)
    files = glob.glob(pathToFiles + '*/*/', recursive=True)
    return files

def extractFramesToFile (pathToFile, dest):

    vidcap = cv2.VideoCapture(pathToFile)
    frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    success = True
    
    for i in range(0, frames):
        success,image = vidcap.read()
        if success:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(dest + "frame" + str(i) + ".jpg", gray)     # save frame as JPEG file
            if cv2.waitKey(10) == 27:                     # exit if Escape is hit
                break
    # When everything done, release the capture
    vidcap.release()
    cv2.destroyAllWindows()

In [60]:
directoriesList = getAllDirectories(pathToVideos)

In [61]:
# It was used to get frames from video and save them as images
# for path in directoriesList:
#     innerFiles = glob.glob(path + '/*')
#     for j, innerPath in enumerate(innerFiles):
#         folderName = innerPath.rsplit('/', 1)[1][:-4]
#         writePath = os.path.join(path, folderName + "/")
#         os.mkdir(writePath)
#         extractFramesToFile(innerPath, writePath)

In [62]:
filesList = getAllFiles(pathToVideos)

/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/


In [64]:
import time
from DE import *
def processSign(framesPath, n):
    
    stop_iteration = 10
    number_candidates = 10
    scale = 0.9
    crossover = 0.6
    
    framesQnt = len(glob.glob(framesPath + "*.jpg"))
    if (framesQnt < 1):
        print("Corrupted video:" + framesPath)
        return -1
        
    de = DifferentialEvolution(n, scale, crossover, number_candidates, stop_iteration)

    print(f"Starting routine for file {framesPath} with n = {n}")
    print("Quantity of frames: " + str(framesQnt))

    de.initialize_NP(framesQnt, framesPath)
    for GENERATION in range(stop_iteration):
        for j in range(number_candidates):
            de.mutation(j, framesQnt, framesPath)
            de.crossover(j, framesPath)
            de.selection(j)
            
    best = de.bestParent()
    resultPath = os.path.join(framesPath, f"DEResult_{n}/")
    os.mkdir(resultPath)
    for frame_number in best[:-1]:
        shutil.copyfile(framesPath + "frame" + str(frame_number) + ".jpg", resultPath + "frame" + str(frame_number) + ".jpg")
    return best

In [65]:
import multiprocessing as mp
from functools import partial

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s [%(levelname)s] - %(message)s',
                    filename="/home/fabiana/Desktop/projeto-final-src/KeyFramesExtraction/keyframes.log",
                    filemode='a')



n_frames = [5, 10, 15]
for n in n_frames:
    logging.info(f"Starting routine for keyframes extraction with n = {n}")
    start_time = time.time()
    mp.freeze_support()
    p = mp.Pool() # default is a number of processes equal to the number of CPU cores
    func = partial(processSign, n=n)
    result_list = p.map(func, filesList)
    p.close()
    p.join()
    print(f"Execution time for routine with n = {n} -> {time.time() - start_time}")
    logging.info(f"Execution time for routine with n = {n} -> {time.time() - start_time}")

Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/B/BISPO/ with n = 15
Quantity of frames: 54
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/D/DOIS1/ with n = 15
Quantity of frames: 49
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/B/BALBUCIAR2/ with n = 15
Quantity of frames: 56
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/P/PONTAPE/ with n = 15
Quantity of frames: 37
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/B/BIOLOGO/ with n = 15
Quantity of frames: 49
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/B/BOLSO/ with n = 15
Quantity of frames: 45
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/D/DESFILAR/ with n = 15
Quantity of frames: 54
Starting routine for file /home/fabiana/Desktop/projeto-final-src/Crawler/Videos/P/PAISAGEM/ with n = 15
Quantity